In [1]:
import tomllib
import numpy as np
from pprint import pprint
import librosa
import requests
from pydub import AudioSegment
from io import BytesIO
from SpotifyTrack import SpotifyTrack
from poc_test import estimate_tempo
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import scipy
import scipy.stats

with open("cfg.toml", "rb") as cfg:
    keys = tomllib.load(cfg)["spotify"]
    c_id = keys["client_id"]
    c_secret = keys["client_secret"]
    auth_manager = SpotifyClientCredentials(client_id=c_id, client_secret=c_secret)
    
sp = spotipy.Spotify(auth_manager=auth_manager)

In [2]:
# The Weeknd's "After Hours" 4yP0hdKOZPNshxUOjY0cZj
#(album_name, album_uri) = ('harvest','spotify:album:7hIjNhXoPqyARBpaKpS3nk')
# (album_name, album_uri) = ('sdp_my_eyes', 'spotify:album:2nhThWvrfDbjXvRfuwlsCE')
(album_name, album_uri) = ('SOS', 'spotify:album:07w0rG5TETcyihsEIZR3qG')
# (album_name, album_uri) = ('avril_lavigne', 'spotify:album:3zXjR3y2dUWklKmmp6lEhy')
# (album_name, album_uri) = ('travy_patty', 'spotify:album:18NOKLkZETa4sWwLMIm0UZ')

tracks_raw = sp.album_tracks(album_id=album_uri.split(':')[-1], market='CA')

tracks = [SpotifyTrack(**(tracks_raw['items'][i])) for i in range(len(tracks_raw['items']))]

In [3]:
#display raw track metadata
tracks_raw

{'href': 'https://api.spotify.com/v1/albums/07w0rG5TETcyihsEIZR3qG/tracks?offset=0&limit=50&market=CA',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7tYKF4w9nC0nq9CsPZTHyP'},
     'href': 'https://api.spotify.com/v1/artists/7tYKF4w9nC0nq9CsPZTHyP',
     'id': '7tYKF4w9nC0nq9CsPZTHyP',
     'name': 'SZA',
     'type': 'artist',
     'uri': 'spotify:artist:7tYKF4w9nC0nq9CsPZTHyP'}],
   'disc_number': 1,
   'duration_ms': 117773,
   'explicit': True,
   'external_urls': {'spotify': 'https://open.spotify.com/track/5xMw6qCcpd2gBXPGTegC4W'},
   'href': 'https://api.spotify.com/v1/tracks/5xMw6qCcpd2gBXPGTegC4W',
   'id': '5xMw6qCcpd2gBXPGTegC4W',
   'is_playable': True,
   'name': 'SOS',
   'preview_url': 'https://p.scdn.co/mp3-preview/0d91430a4a328573f8e4630bdca40d9d9b1dc160?cid=970a76441550417c8d7779d78038bc31',
   'track_number': 1,
   'type': 'track',
   'uri': 'spotify:track:5xMw6qCcpd2gBXPGTegC4W',
   'is_local': False},
  {'artists': [{'externa

In [4]:
#display spotify track objects were created in memory
tracks

In [5]:
#save the spotify wavs from the album
def save_wavs(tracks):
    for track in tracks:
        r = requests.get(track.preview_url)
        audio = AudioSegment.from_mp3(BytesIO(r.content))
        wav_path = f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav'
        audio.export(wav_path, format='wav')

In [6]:
save_wavs(tracks)

In [ ]:
#dynamic programing solution for beat tracking reverse engineering
#dynamic programming implementation of beat tracking - converted and adapted code from 
# https://www.tandfonline.com/doi/epdf/10.1080/09298210701653344?needAccess=true, librosa documentation. 

def compute_localscore(onset_envelope, period):
    window = np.exp(-0.5 *(np.arange(-period, period+1) * 32/period)**2)
    #I LOVE CONVOLUTION
    normalized_onset = onset_envelope/onset_envelope.std(ddof = 1)
    #print("ConVOLUTING!!1")
    return scipy.signal.convolve(normalized_onset, window, "same")

def dynamic_compute(local_score, period, tightness):
    #initiate dp arrays with zeroes
    backlink = np.zeros_like(local_score, dtype = int)
    cumulative_score = np.zeros_like(local_score)

    #define search range. Will be iteratively increased 
    window = np.arange(-2*period, -np.round(period/2)+1, dtype=int)

    tx_cost = -tightness * (np.log(-window/period) **2)
    first_beat = True
    for i, score_i in enumerate(local_score):
        #force range to not go past t = 0
        z_pad = np.maximum(0, min(-window[0], len(window)))
        scores = tx_cost.copy()
        scores[z_pad:] = scores [z_pad:]+cumulative_score[window[z_pad:]]
        beat_location = np.argmax(scores) #compute index of max
        cumulative_score[i] = score_i + scores[beat_location]

        #special case for intiation:
        if first_beat == True and score_i < 0.001*local_score.max(): 
            backlink[i] = -1
        else: 
            backlink[i] = window[beat_location]
            first_beat = False
        
        #update window range iteratively
        window += 1
    return backlink, cumulative_score

def compute_tempo(audio_file_path, hop_len=512, tightness=8.0, initial_bpm = 75):
    #initiate some variables for the beat tracking. we compute beats, then go to the global BPM
    #tightness - float scalar for how closely it adheres to bpm, default to 100. Seems like lower values works better?
    #audio_file_path is a pointer to the .wav of the audio, instead of the MP3 file

    #compute the onset from the provided audio file, compute starting BPM
    signal, sampling_rate = librosa.load(audio_file_path, sr=None, mono=True)
    print(f"sampling rate: {sampling_rate}")
    onset_envelope = librosa.onset.onset_strength(y=signal, sr = sampling_rate, hop_length=hop_len)

    #compute resolution of the fast fourrier transform
    fft_res = sampling_rate/hop_len
    period = np.round(60.0 *fft_res/initial_bpm)

    #compute automatic gain control of the onset envelope
    local_score = compute_localscore(onset_envelope, period)
    #dynamic programming time

    backlink, cumulative_score = dynamic_compute(local_score, period, tightness)
    #get the pos of the last beat
    #need to first compute the median score, and local maxes. return local maxes afterwards
    local_max_scores = librosa.util.localmax(cumulative_score)
    median_score = np.median(cumulative_score[np.argwhere(local_max_scores)])
    last_beat_location = np.argwhere((cumulative_score*local_max_scores*2>median_score)).max()

    beats = [last_beat_location]

    while backlink[beats[-1]] >= 0: beats.append(backlink[beats[-1]])

    # Put the beats in ascending order
    # Convert into an array of frame numbers
    beats = np.array(beats[::-1], dtype=int)

    #
    first_beat_time, last_beat_time = librosa.frames_to_time((beats[0],beats[-1]),sr=sampling_rate)
    tempo = 60/((last_beat_time-first_beat_time)/(len(beats)-1))
    return tempo

#Testing time
#print(f"Dynamic Programming BPM method computed: {compute_tempo("mp3s/505_artic.mp3", tightness = 100.0, initial_bpm=120)}")

In [8]:
DP_method_tempos = [compute_tempo(f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav').item() for track in tracks]

sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100
sampling rate: 44100


In [9]:
#get spotify's tempos
acc_tempos = [sp.audio_features(harvest_track.id)[0].get('tempo') for harvest_track in tracks]

In [10]:
pd.DataFrame(zip(DP_method_tempos, acc_tempos),
             index=(t.name for t in tracks),
             columns=['OurAlgoTempo', 'SpotifyTempo'])

,OurAlgoTempo,SpotifyTempo
SOS,1033.59375,119.159
Kill Bill,1033.59375,88.993
Seek & Destroy,1033.59375,152.069
Low,1033.59375,145.044
Love Language,1033.59375,65.038
Blind,1033.59375,114.225
Used (feat. Don Toliver),1033.59375,150.015
Snooze,1033.59375,143.008
Notice Me,1033.59375,159.919
Gone Girl,1033.59375,150.093
